In [78]:
from collections import Counter

def calculate_sparse_and_topN_accuracy(dense_result, mrc, are_texts_similar, topk=[1,2,3,5,10,15,20]):
    # 각 top N에 정답이 몇 개 있는지 카운팅할 Counter
    correct_topN_counter = Counter()

    # 전체 row 수
    total_rows = len(dense_result)
    total_cols = len(dense_result.columns)

    # dense_result의 모든 row에 대해 반복
    for idx, row in dense_result.iterrows():
        mrc_id = row['id']  # 각 row의 id 추출

        # mrc에서 해당 id의 context 추출
        try:
            mrc_context = mrc[mrc['id'] == mrc_id]['context'].values[0]
        except IndexError:
            print(f"Context not found for ID: {mrc_id}")
            continue  # 만약 mrc에서 id가 없으면 다음 row로 넘어감

        # topN_context들 중에서 mrc_context와 일치하는 index를 찾음
        for i in range(1, total_cols-1):  # top1_context부터 top100_context까지 확인
            top_context = row[f'top{i}_context']

            # 공백 및 줄바꿈 처리
            mrc_context = mrc_context.strip().replace("\n\n", " ").replace("\n", " ")
            top_context = top_context.strip().replace("\n\n", " ").replace("\n", " ")

            # 유사도 비교
            if are_texts_similar(mrc_context, top_context):
                correct_topN_counter[i] += 1  # 해당하는 topN에 카운트 추가
                break

    # Sparse accuracy 계산 및 출력
    sparse_accuracy = (sum(correct_topN_counter.values()) / total_rows) * 100
    print(f"Total Count: {total_rows}")
    print(f"Missing Count: {total_rows - sum(correct_topN_counter.values())}")
    print(f"Top 100 Sparse accuracy: {sparse_accuracy:.2f}%")

    # 각 topN에 대한 정확도 출력
    print("\nAccuracy Results:")
    for top_n in topk:  # top1부터 top5까지 정확도 확인
        correct_count = sum([count for n, count in correct_topN_counter.items() if n <= top_n])
        accuracy = (correct_count / total_rows) * 100
        print(f"Top {top_n} accuracy: {accuracy:.2f}% ({correct_count}/{total_rows})")

In [66]:
from difflib import SequenceMatcher

def are_texts_similar(text1, text2, threshold=0.95, print_score=False):
    """
    두 텍스트가 주어진 유사도 threshold 이상으로 유사한지 확인하는 함수.
    
    Parameters:
        text1 (str): 첫 번째 텍스트
        text2 (str): 두 번째 텍스트
        threshold (float): 유사도를 판단하는 기준 (0 ~ 1 사이 값, 기본값은 0.9)

    Returns:
        bool: 유사도가 threshold 이상이면 True, 그렇지 않으면 False
    """
    
    # 불필요한 공백, 줄바꿈 등을 제거
    text1_clean = text1.strip().replace("\n\n", " ").replace("\n", " ")
    text2_clean = text2.strip().replace("\n\n", " ").replace("\n", " ")

    # 유사도 계산 함수
    def similarity(a, b):
        return SequenceMatcher(None, a, b).ratio()

    # 유사도 계산
    similarity_score = similarity(text1_clean, text2_clean)
    if print_score:
        print(f"Similarity: {similarity_score * 100:.2f}%")

    # 유사도가 threshold 이상이면 True 반환
    return similarity_score >= threshold

In [35]:
import json
import pandas as pd
from datasets import load_from_disk


data = json.load(open('../data/raw/wikipedia_documents.json'))
wiki = pd.DataFrame(data).T

dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])

mrc = pd.concat([train_df, valid_df])

In [84]:
import json
import pandas as pd

BGEm3ko_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_BGEm3ko_max.csv')
gteMultibase = pd.read_csv('../data/pipeline/BM25Ensemble_top100_gteMultibase_max.csv')
KoE5_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_KoE5_max.csv')
KoE5_mean = pd.read_csv('../data/pipeline/BM25Ensemble_top100_KoE5_mean.csv')
original = pd.read_csv('../data/pipeline/BM25Ensemble_top100_original.csv')
ParaMiniLM_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_ParaMiniLM_max.csv')
# qwen_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_Qwen_max.csv')
upskyyBGEm3korean_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_upskyyBGEm3korean_max.csv')
qwen2_1_5b_max = pd.read_csv('../data/pipeline/BM25Ensemble_top100_Qwen2-1_5b_max.csv')

In [133]:
bge_reranker = pd.read_csv('../data/pipeline/BM25Ensemble_top100_bge-reranker.csv')

In [135]:
topk = [1,2,3,5,10,15,20,50,100]
calculate_sparse_and_topN_accuracy(bge_reranker, mrc, are_texts_similar, topk=topk)
print('*'*50)
# calculate_sparse_and_topN_accuracy(gteMultibase, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(ParaMiniLM_max, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(original, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(upskyyBGEm3korean_max, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(BGEm3ko_max, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(KoE5_max, mrc, are_texts_similar, topk=topk)
# print('*'*50)
# calculate_sparse_and_topN_accuracy(KoE5_mean, mrc, are_texts_similar, topk=topk)


Total Count: 240
Missing Count: 5
Top 100 Sparse accuracy: 97.92%

Accuracy Results:
Top 1 accuracy: 91.25% (219/240)
Top 2 accuracy: 95.42% (229/240)
Top 3 accuracy: 95.83% (230/240)
Top 5 accuracy: 96.67% (232/240)
Top 10 accuracy: 97.08% (233/240)
Top 15 accuracy: 97.50% (234/240)
Top 20 accuracy: 97.50% (234/240)
Top 50 accuracy: 97.92% (235/240)
Top 100 accuracy: 97.92% (235/240)
**************************************************


In [125]:
BGEm3ko_max_test = pd.read_csv('../data/pipeline/test_BM25Ensemble_top100_BGEm3ko_max.csv')
KoE5_max_test = pd.read_csv('../data/pipeline/test_BM25Ensemble_top100_KoE5_max.csv')
original_test = pd.read_csv('../data/pipeline/test_BM25Ensemble_top100_original.csv')


In [130]:
import pandas as pd
from collections import Counter

# 여러 dataframe을 받아 hard voting 수행
def hard_voting(df_list):
    result_rows = []  # 결과를 저장할 리스트
    
    # 각 query마다 처리
    for idx in range(len(df_list[0])):
        id_val = df_list[0].iloc[idx]['id']
        question_val = df_list[0].iloc[idx]['question']
        chosen_contexts = []  # 각 query에 대해 선택된 topn_context를 저장할 리스트

        # top1_context부터 top100_context까지 처리
        for top_n in range(1, 101):
            candidates = []

            # 모든 dataframe에서 해당 query의 topn_context 후보 수집
            for df in df_list:
                candidates.append(df.iloc[idx][f'top{top_n}_context'])

            # 다수결로 가장 많이 나온 후보 선택 (중복된 문서 제외)
            filtered_candidates = [c for c in candidates if c not in chosen_contexts]
            if len(filtered_candidates) == 0:
                chosen_context = None
            else:
                chosen_context = Counter(filtered_candidates).most_common(1)[0][0]

            chosen_contexts.append(chosen_context)

        # 결과에 저장
        result_rows.append([id_val, question_val] + chosen_contexts)
    
    # 리스트로 모은 결과를 DataFrame으로 변환
    result = pd.DataFrame(result_rows, columns=df_list[0].columns)
    
    return result

# hard voting 실행
# df_list = [BGEm3ko_max, gteMultibase, KoE5_max, KoE5_mean, original, ParaMiniLM_max, upskyyBGEm3korean_max]
df_list = [BGEm3ko_max_test, KoE5_max_test, original_test]
final_df = hard_voting(df_list)
# save final_df
# final_df.to_csv('../data/pipeline/test_BM25Ensemble_top100_hard_voting.csv', index=False)

x = 20
final_df_cut = final_df[['id', 'question'] + [f'top{i}_context' for i in range(1, x+1)]]
# check is null
print(final_df_cut.isnull().sum().sum())
calculate_sparse_and_topN_accuracy(final_df_cut, mrc, are_texts_similar, topk=topk)

13
Context not found for ID: mrc-1-000653
Context not found for ID: mrc-1-001113
Context not found for ID: mrc-0-002191
Context not found for ID: mrc-0-003951
Context not found for ID: mrc-1-001272
Context not found for ID: mrc-1-000993
Context not found for ID: mrc-0-005021
Context not found for ID: mrc-1-000163
Context not found for ID: mrc-0-001283
Context not found for ID: mrc-0-004543
Context not found for ID: mrc-0-000439
Context not found for ID: mrc-0-002895
Context not found for ID: mrc-0-000535
Context not found for ID: mrc-1-001724
Context not found for ID: mrc-0-000901
Context not found for ID: mrc-0-001606
Context not found for ID: mrc-0-000266
Context not found for ID: mrc-0-001326
Context not found for ID: mrc-0-000032
Context not found for ID: mrc-0-005215
Context not found for ID: mrc-0-005407
Context not found for ID: mrc-0-003683
Context not found for ID: mrc-0-003644
Context not found for ID: mrc-0-002835
Context not found for ID: mrc-0-000049
Context not found for 

In [129]:
final_df.shape

(600, 102)

In [137]:
df = pd.read_csv('../data/pipeline/test_BM25Ensemble_top100_original.csv')
df.shape

(600, 102)

In [154]:
# split row 600 to 200,200,200
# split column 0~47
df1 = df.iloc[:100, :47]
df2 = df.iloc[100:200, :47]
df250 = df.iloc[200:250, :47]
df300 = df.iloc[250:300, :47]
df350 = df.iloc[300:350, :47]
df400 = df.iloc[350:400, :47]
df450 = df.iloc[400:450, :47]
df500 = df.iloc[450:500, :47]
df550 = df.iloc[500:550, :47]
df600 = df.iloc[550:, :47]


df1.shape, df2.shape, df250.shape, df300.shape, df350.shape, df400.shape, df450.shape, df500.shape, df550.shape, df600.shape
# # # save
# df1.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_1.csv', index=False)
# df2.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_2.csv', index=False)
# df3.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_3.csv', index=False)
# df4.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_4.csv', index=False)
# df5.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_5.csv', index=False)
# df6.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_6.csv', index=False)
df250.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_200_250.csv', index=False)
df300.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_250_300.csv', index=False)
df350.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_300_350.csv', index=False)
df400.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_350_400.csv', index=False)
df450.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_400_450.csv', index=False)
df500.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_450_500.csv', index=False)
df550.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_500_550.csv', index=False)
df600.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_550_600.csv', index=False)

In [152]:
# split row 600 to 200,200,200
# split column 0~47
df1 = df.iloc[:100, :47]
df2 = df.iloc[100:200, :47]
df3 = df.iloc[200:300, :47]
df4 = df.iloc[300:400, :47]
df5 = df.iloc[400:500, :47]
df6 = df.iloc[500:600, :47]


df1.shape, df2.shape, df3.shape, df4.shape, df5.shape, df6.shape
# # # save
df1.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_1.csv', index=False)
df2.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_2.csv', index=False)
df3.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_3.csv', index=False)
df4.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_4.csv', index=False)
df5.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_5.csv', index=False)
df6.to_csv('../data/pipeline/test_BM25Ensemble_top45_original_6.csv', index=False)

In [147]:
df1.columns

Index(['id', 'question', 'top1_context', 'top2_context', 'top3_context',
       'top4_context', 'top5_context', 'top6_context', 'top7_context',
       'top8_context', 'top9_context', 'top10_context', 'top11_context',
       'top12_context', 'top13_context', 'top14_context', 'top15_context',
       'top16_context', 'top17_context', 'top18_context', 'top19_context',
       'top20_context', 'top21_context', 'top22_context', 'top23_context',
       'top24_context', 'top25_context', 'top26_context', 'top27_context',
       'top28_context', 'top29_context', 'top30_context', 'top31_context',
       'top32_context', 'top33_context', 'top34_context', 'top35_context',
       'top36_context', 'top37_context', 'top38_context', 'top39_context',
       'top40_context', 'top41_context', 'top42_context', 'top43_context',
       'top44_context', 'top45_context'],
      dtype='object')

In [150]:
test_reranker1 = pd.read_csv('../data/pipeline/test_BM25Ensemble_top45_bge_reranker_1.csv')
print(test_reranker1.shape)
test_reranker1

(100, 47)


,id,question,top1_context,top2_context,top3_context,top4_context,top5_context,top6_context,top7_context,top8_context,...,top36_context,top37_context,top38_context,top39_context,top40_context,top41_context,top42_context,top43_context,top44_context,top45_context
0,mrc-1-000653,유령'은 어느 행성에서 지구로 왔는가?,별하늘계의 변경에 위치한 외계행성 사만(サマーン) 출신의 여자아이. 지구 나이로는 ...,더크 젠틀리의 성스러운 탐정사무소의 줄거리는 이야기의 중추적인 부위에 자리잡은 시간...,주인공인 비론 파릴(Biron Farrill)은 네페로스 행성의 지도자의 아들로서 ...,영화를 시작할 때 해설자는 돌고래가 지구가 철거될 것을 알고 있었다고 설명한다. 돌...,은하수를 여행하는 히치하이커를 위한 안내서는 방대한 지식의 은하대백과사전보다 더 유...,※설명은 애니 속 내용 및 인터넷에 돌아다니는 정보들을 모아 서술됨..\n\n;카카...,모린의 직업은 퍼스널 쇼퍼이다. 파리에 머물며 유명 모델인 키라의 옷이나 액세서리를...,"유상론(有想論)의 문자 그대로의 뜻은 '생각이 존재한다는 견해 또는 이론'인데, 이...",...,우주선(宇宙船)은 우주를 다니도록 만들어진 탈것 또는 장치이다. 준궤도 우주비행에서...,"중국 송나라 때 황명으로 편찬되었으며, 해인사 팔만대장경으로 전해지는 경덕전등록에 ...",메신저 (우주선)\n메신저는 NASA에서 발사한 수성 탐사선이다. 메신저(MESSE...,현용수(Yong-Soo Hyun) 박사는 4000년 동안 성공한 민족으로 검증된 유...,"기상(氣象)은 강수, 바람, 구름 등 대기 중에서 일어나는 각종 물리적인 현상을 통...","소크라테스 이전 철학자(독일어: Vorsokratiker, 영어: Presocrat...",이 인터뷰는 2019년 12월 11일 육군사관학교 군사사학과 나종남 교수와 진행하였...,"쾌락계산법(Felicific calculus)은 공리주의, 공익주의를 제창한 제레미...",경제학상에 대한 논란은 다음과 같은 문제들에서 발생한다.\n# 노벨의 유언에 명시되...,"법정은 다음과 같이 평결하였다.\n\n:# 미국 정부(및 호주, 뉴질랜드, 한국 정..."
1,mrc-1-001113,용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?,냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생겨나고 강대국들의 ...,"PC판은 소리가 수록되어 있지 않기 때문에, PC판 이외의 성우를 기재.\n\n; ...",클리블랜드에 기반을 둔 선박 제조업자 조지 스타인브레너(1930년 ~ 2010년)가...,NXT에서 알레이스터 블랙(Aleister Black)이라는 새로운 링네임으로 NX...,1989년 퍼시픽 리그 페넌트레이스는 오릭스 브레이브스 가 개막 8연승으로 순조롭게...,왕양명의 일생은 대략 다섯 시기로 나누어 볼 수 있다. 일본 학자 콘도 야스노부[近...,"《잡아함경》에는 무명(無明)에 대한 정의와 마음(6식 또는 8식, 즉 심왕, 즉 심...",독일은 자국 대표팀이 월드컵에서 좋은 성적을 거둔 경우가 많아 자국 대표팀에 자부심...,...,"2005년(제56회 / 지바 롯데 마린스 대 한신 타이거스 1차전·10월 22일, ...",그리샤는 2008년에 FIFA 국제 심판이 됐다. \n\n그리샤는 2012년 아프리...,1940년 1월 4일 국민정부 시대 중화민국 장시성 간저우 시에서 출생하였으며 지난...,"소코는 나바라 지방 가르데 출신으로, 지역의 아마추어 구단인 오베레나에서 뛰다가 오...","추크 제도에 인류가 최초로 정착하게 된 것이 언제인지는 밝혀지지 않았지만, 문헌과 ...",1998년의 네덜란드 대 대한민국(Wereldkampioenschap voetbal...,메신저 (우주선)\n메신저는 NASA에서 발사한 수성 탐사선이다. 메신저(MESSE...,보문산공원 진입로 우측에 있는 정려각에 보관된 송두원과 그의 할머니 윤씨부인의 정려...,"중풍(中風, palsy)은 다양한 유형의 마비를 가리키는 의학 용어이며 뇌의 혈관...",베피콜롬보\n베피콜롬보는 수성 탐사 계획 중 하나로 ESA와 JAXA가 공동으로 계...
2,mrc-0-002191,돌푸스에게 불특정 기간동안 하원이 잠시 쉬는 것을 건의 받았던 인물은?,"1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거졌다. 당시 오스트리아 ...","들어가며\n\n16세기 종교개혁을 대표하고 있는 루터, 츠빙글리 그리고 칼뱅 옆에서...",유럽에서 제1차 세계 대전의 공포와 그 직후의 시기를 목격한 많은 자들과 마찬가지로...,1844년 7월 28일 런던 근교의 스트랫퍼드에서 태어났다. 그는 여덟 남매 가운데...,"918년 7월 24일(음력 6월 을묘일), 궁예의 숙청에 위기의식과 반감을 느낀 신...","항상 극단적으로 활동적이었다. 긴 기간동안을 일하거나, 쉬는 동안에도 몇 가지 스포...","200여 년 전에 박예진이 봉산리 점촌마을로 들어와 옹기를 굽기 시작한 이래, 6대...",그는 불가리아 축구 국가대표팀의 주장으로 우아한 축구를 하는 것으로 유명한 공격수이...,...,"크리스틴 스베리스도티르(Kristín Sverrisdóttir, Kristin Sv...",2008년 4월 21일 서울 용산구 이태원동에서 70대 노인과 실랑이를 벌이다 폭행...,근저당(根抵當)은 계속적인 거래관계로부터 장래 생기게 될 다수의 불특정 채권을 담보...,"창업자는 전직 영어강사였던 마윈 회장으로, 인터넷의 시대가 열릴 것을 기대하고 각종...",독일 연방정보부 저널리스트 스캔들은 여러 명의 기자들이 독일 연방정보부원에게 타 기...,대한민국 윤락행위등방지법\n윤락행위등방지법(淪落行爲－等防止法)은 선량한 풍속을 해치...,금전신탁의 운용방법은 특정한 특정(特定) 금전신탁과 특정하지 않은 불특정(不特定)금...,제639조 (타인을 위한 보험) (1) 보험계약자는 위임을 받거나 위임을 받지 아니...,러시아 해킹 그룹 이름에 따라붙는 APT(Advance Persistent Thre...,유상증자는 쉽게 말하면 기업이 돈이 필요할 때 신규로 자기 회사 주식을 발행해 그 ...
3,mrc-0-003951,"마오리언어와 영어, 뉴질랜드 수화를 공식 언어로 사용하는 나라는?","유럽인들의 아메리카와 오세아니아 식민지화로 인해 아메리카와 오세아니아의 문화적, 민...",마오리어를 쓰는 나라:\n* 뉴질랜드\n마셜어를 쓰는 나라:\n* 마셜 제도\n마케...,"영어는 발음, 어휘, 철자법 등이 영국, 북미, 영어를 사용하는 아프리카 국가, 싱...","농아인 공동체는 멕시코 수화와 유카탄 수화를 사용하고, 바하칼리포르니아주 북부에서는...",언어 목록모어 화자 수순 언어 목록\n\n자연어 중에 세계적으로 많은 인구가 사용하...,"다중심언어 또는 복수중심언어(pluricentric language, polycen...","다중심언어로 분류되는 언어들은, 국경을 넘어서 여러 국가 또는 지역에서 사용되며, ...",멕시코의 언어는 다양하고 그 중 스페인어가 가장 널리 쓰인다. 멕시코 원주민 언어들...,...,바운티 호의 반란 이후 반란 선원들은 타히티인들을 데려와 자급적인 생활을 꾸려갔으며...,"도시의 중심에 와이카토 강이 흐르며, 이를 기준으로 동부와 서부지역(뉴질랜드 국회의...",오클랜드시티(Auckland City)는 뉴질랜드 북섬에 있는 오클랜드의 중심 도시...,와이탕이 조약\n1840년 영국 해군 장교 윌리엄 홉슨이 총독으로 뉴질랜드를 찾아온...,"쿠루흐어( कुड़ुख़, Kurukh /ˈkʊrʊx/)는 자르칸드주, 마디아프라데...","스라난 통고의 어휘는 영어, 네덜란드어, 포르투갈어, 중부와 서부 아프리카어에서 비...","현재, 수리가오논 족은 북수리가오 주와 남수리가오 주, 그리고 북아구산 주의 화자를...","AGROVOC은 Agriculture, Fo